In [ ]:
import pandas as pd
import requests
import math

def calculate_distance(lat1, lon1, lat2, lon2):
    # Function to calculate distance between two points using Haversine formula
    radius = 3959.0  # Radius of the Earth in miles

    lat1 = math.radians(lat1)
    lon1 = math.radians(lon1)
    lat2 = math.radians(lat2)
    lon2 = math.radians(lon2)

    dlon = lon2 - lon1
    dlat = lat2 - lat1

    a = math.sin(dlat/2)**2 + math.cos(lat1) * math.cos(lat2) * math.sin(dlon/2)**2
    c = 2 * math.atan2(math.sqrt(a), math.sqrt(1 - a))
    distance = radius * c
    return distance

user_address = input("Enter your address: ")

# Replace 'YOUR_OPENCAGE_API_KEY' with your actual OpenCage Data API key
api_key = 'f8eb8e6d2a26423f9cb530bdc39c6e45'

# Define the OpenCage Data Geocoding API endpoint
api_endpoint = 'https://api.opencagedata.com/geocode/v1/json'

# Define the parameters for the geocoding request
params = {
    'q': user_address,
    'key': api_key,
}

# Send the geocoding request to OpenCage Data API
response = requests.get(api_endpoint, params=params)

if response.status_code == 200:
    data = response.json()
    if data.get('results'):
        # Extract latitude and longitude from the response
        first_result = data['results'][0]
        user_latitude = float(first_result['geometry']['lat'])
        user_longitude = float(first_result['geometry']['lng'])

        # Load the nightclub dataset with 'ISO-8859-1' encoding
        nightclubs = pd.read_csv('nightclubs.csv', encoding='ISO-8859-1')

        # Calculate distances between user location and nightclubs in miles
        nightclubs['Distance (Miles)'] = nightclubs.apply(
            lambda row: calculate_distance(user_latitude, user_longitude, float(row['Latitude']), float(row['Longitude'])),
            axis=1
        )

        # Round distances to 2 decimal places
        nightclubs['Distance (Miles)'] = nightclubs['Distance (Miles)'].round(2)

        # Display the ratings of the nearest nightclubs along with their names and distances
        nearest_nightclubs = nightclubs.nsmallest(10, 'Distance (Miles)')[['Name', 'Rating', 'Distance (Miles)']]
        print(nearest_nightclubs)


In [1]:
import pandas as pd
import requests
import math
import sys
from PyQt5.QtWidgets import QApplication, QWidget, QLabel, QLineEdit, QPushButton, QVBoxLayout, QHBoxLayout, \
    QTableWidget, QTableWidgetItem
from PyQt5.QtGui import QLinearGradient, QColor, QPalette, QBrush
from PyQt5.QtCore import Qt
from sklearn.cluster import KMeans

def calculate_distance(lat1, lon1, lat2, lon2):
    # Function to calculate distance between two points using Haversine formula
    radius = 3959.0  # Radius of the Earth in miles

    lat1 = math.radians(lat1)
    lon1 = math.radians(lon1)
    lat2 = math.radians(lat2)
    lon2 = math.radians(lon2)

    dlon = lon2 - lon1
    dlat = lat2 - lat1

    a = math.sin(dlat / 2) ** 2 + math.cos(lat1) * math.cos(lat2) * math.sin(dlon / 2) ** 2
    c = 2 * math.atan2(math.sqrt(a), math.sqrt(1 - a))
    distance = radius * c
    return distance

class NightclubRecommenderApp(QWidget):
    def __init__(self):
        super().__init__()

        self.setWindowTitle("Nightclub Recommender")
        self.setGeometry(100, 100, 1920, 1080)

        # Placeholder paths and colors
        self.background_image_path = "Clubbed.jpg"  # Replace with the path to your image
        self.gradient_color1 = QColor(255, 150, 150)  # Light Red
        self.gradient_color2 = QColor(150, 150, 255)  # Light Blue

        self.init_ui()

    def init_ui(self):
        self.create_gradient_background()

        # Input Widgets
        self.label_address = QLabel("Enter your address:")
        self.entry_address = QLineEdit(self)
        self.label_food = QLabel("Enter your food preference:")
        self.entry_food = QLineEdit(self)
        # Fixed number of clusters
        num_clusters = 3
        self.label_clusters = QLabel(f"Number of clusters (fixed): {num_clusters}")
        self.btn_recommend = QPushButton("Get Recommendations", self)
        self.btn_recommend.clicked.connect(self.get_recommendations)

        # Output Table
        self.table = QTableWidget(self)
        self.table.setColumnCount(4)  # Columns: Name, Rating, Distance, Food
        self.table.setHorizontalHeaderLabels(["Name", "Rating", "Distance (Miles)", "Food"])

        # Result Text
        self.result_text = QLabel(self)
        self.result_text.setAlignment(Qt.AlignCenter)
        self.result_text.setStyleSheet("color: red;")  # Adjust styling as needed

        # Exit Button
        self.btn_exit = QPushButton("Exit", self)
        self.btn_exit.clicked.connect(self.close)
        self.btn_exit.setStyleSheet("background-color: #FF0000; color: white;")

        # Layout
        input_layout = QVBoxLayout()
        input_layout.addWidget(self.label_address)
        input_layout.addWidget(self.entry_address)
        input_layout.addWidget(self.label_food)
        input_layout.addWidget(self.entry_food)
        input_layout.addWidget(self.label_clusters)
        input_layout.addWidget(self.btn_recommend)
        input_layout.addStretch()

        output_layout = QVBoxLayout()
        output_layout.addWidget(self.table)
        output_layout.addWidget(self.result_text)
        output_layout.addStretch()

        buttons_layout = QHBoxLayout()
        buttons_layout.addStretch()
        buttons_layout.addWidget(self.btn_exit)

        main_layout = QHBoxLayout()
        main_layout.addLayout(input_layout)
        main_layout.addLayout(output_layout)
        main_layout.addLayout(buttons_layout)

        self.setLayout(main_layout)

    def create_gradient_background(self):
        gradient = QLinearGradient(0, 0, 0, self.height())
        gradient.setColorAt(0, self.gradient_color1)
        gradient.setColorAt(1, self.gradient_color2)

        palette = self.palette()
        
        # Use setBrush method with QPalette.Window role
        palette.setBrush(QPalette.Window, QBrush(gradient))
        
        self.setPalette(palette)

    def get_recommendations(self):
        user_address = self.entry_address.text()
        user_food_preference = self.entry_food.text()
        # Fixed number of clusters
        num_clusters = 3

        # Replace 'YOUR_OPENCAGE_API_KEY' with your actual OpenCage Data API key
        api_key = 'f8eb8e6d2a26423f9cb530bdc39c6e45'

        # Define the OpenCage Data Geocoding API endpoint
        api_endpoint = 'https://api.opencagedata.com/geocode/v1/json'

        # Define the parameters for the geocoding request
        params = {
            'q': user_address,
            'key': api_key,
        }

        # Send the geocoding request to OpenCage Data API
        response = requests.get(api_endpoint, params=params)

        if response.status_code == 200:
            data = response.json()
            if data.get('results'):
                # Extract latitude and longitude from the response
                first_result = data['results'][0]
                user_latitude = float(first_result['geometry']['lat'])
                user_longitude = float(first_result['geometry']['lng'])

                # Load the nightclub dataset with 'ISO-8859-1' encoding
                nightclubs = pd.read_csv('nightclubs.csv', encoding='ISO-8859-1')

                # Calculate distances between user location and nightclubs in miles
                nightclubs['Distance (Miles)'] = nightclubs.apply(
                    lambda row: calculate_distance(user_latitude, user_longitude, float(row['Latitude']),
                                                   float(row['Longitude'])),
                    axis=1
                )

                # Round distances to 2 decimal places
                nightclubs['Distance (Miles)'] = nightclubs['Distance (Miles)'].round(2)

                # Extract features for clustering
                features = nightclubs[['Latitude', 'Longitude', 'Rating', 'Distance (Miles)']]

                # Apply KMeans clustering
                kmeans = KMeans(n_clusters=num_clusters, random_state=42)
                nightclubs['Cluster'] = kmeans.fit_predict(features)

                # Filter nightclubs based on the cluster and food preference
                recommended_nightclubs = nightclubs[
                    (nightclubs['Cluster'] == nightclubs['Cluster'].mode()[0]) &
                    (nightclubs['Food'].str.lower().str.contains(user_food_preference.lower()))
                ]

                # Sort nightclubs by distance
                recommended_nightclubs = recommended_nightclubs.sort_values(by='Distance (Miles)')

                # Display the recommendations in the table
                if not recommended_nightclubs.empty:
                    self.table.setRowCount(len(recommended_nightclubs))
                    for i, (_, row) in enumerate(recommended_nightclubs.iterrows()):
                        self.table.setItem(i, 0, QTableWidgetItem(row['Name']))
                        self.table.setItem(i, 1, QTableWidgetItem(str(row['Rating'])))
                        self.table.setItem(i, 2, QTableWidgetItem(str(row['Distance (Miles)'])))
                        self.table.setItem(i, 3, QTableWidgetItem(row['Food']))
                    self.result_text.clear()  # Clear any previous messages
                else:
                    self.table.setRowCount(0)  # Clear the table
                    self.result_text.setText("Sorry, no recommendations available for your preferences.")
            else:
                self.table.setRowCount(0)  # Clear the table
                self.result_text.setText("Invalid address. Please try again.")
        else:
            self.table.setRowCount(0)  # Clear the table
            self.result_text.setText("Failed to retrieve location data. Please try again.")

if __name__ == '__main__':
    app = QApplication(sys.argv)
    window = NightclubRecommenderApp()
    window.show()
    sys.exit(app.exec_())


c:\Users\Harsh\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\cluster\_kmeans.py:1416: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  super()._check_params_vs_input(X, default_n_init=10)
c:\Users\Harsh\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\cluster\_kmeans.py:1416: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  super()._check_params_vs_input(X, default_n_init=10)
c:\Users\Harsh\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\cluster\_kmeans.py:1416: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  super()._check_params_vs_input(X, default_n_init=10)
c:\Users\Harsh\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\clus

SystemExit: 0

C:\Users\Harsh\AppData\Roaming\Python\Python312\site-packages\IPython\core\interactiveshell.py:3561: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)
